Importing all the necassary libraries

In [1]:
import numpy as np
import pandas as pd
import os
import math

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Creating a confusion matrix class that is used to evaluate the trained model after testing it on the test set

In [20]:
class confusion_matrix:
  def __init__(self):
    self.true_Real = 0
    self.true_Fake = 0
    self.false_Real = 0
    self.false_Fake = 0
    self.total = 0
    self.right = 0
  
  def update_matrix(self , actual , predicted ):
    self.total += 1
    if actual == predicted:
      self.right += 1
      if actual == 'Positive':
        self.true_Real += 1
      else:
        self.true_Fake += 1

    if actual != predicted:
      if actual == 'Positive':
        self.false_Real += 1
      else:
        self.false_Fake += 1
  
  def results(self):
    p = self.true_Real/(self.true_Real + self.false_Real )
    r = self.true_Real/(self.true_Real + self.false_Fake )
    print( "Accuracy : " , self.right/(self.total )  * 100)
    print( "Precision: " , self.true_Real/(self.true_Real + self.false_Real ) * 100 )
    print( "Recall   : " , self.true_Real/(self.true_Real + self.false_Fake )* 100 )
    print( "F1 score : " , 2* ((p*r)/(p+r)) * 100 )

Reading data from the given training data set and inserting it into their respective classes

In [5]:
classes = ['Negative' , 'Positive']

Train = {}


list_of_df = []

for file_name in os.listdir(r'drive/My Drive/Sentimental Analysis/Urdu Data/'):
  data = pd.read_excel(r'drive/My Drive/Sentimental Analysis/Urdu Data/' + file_name  , names = ['Tweet', 'Class' ])
  list_of_df.append(data)

data = pd.concat(list_of_df, ignore_index=True)

In [6]:
# naming sentiments
data["Class"].replace({'N': "Negative", 'P': "Positive"}, inplace=True)

In [7]:
# Shuffle the dataset
data.dropna(inplace=True)

# Remove unannotated data
data = data.reset_index(drop=True)
data = data.sample(frac=1).reset_index(drop=True)

In [8]:
for index in data.index:
    if "TWEET: RT @" in data.loc[index,'Tweet']:
      cleanString = data.loc[index,'Tweet']
      cleanString = cleanString[ cleanString.find('@') :]
      cleanString = cleanString[ cleanString.find(' ')+1 :]
      data.loc[index,'Tweet'] = cleanString

In [9]:
# Dividing Data into train and test Sets

Split_Ratio = 0.8

Train_df = data.iloc[: int(len(data)*Split_Ratio) , :].reset_index(drop=True)
Test_df = data.iloc[int((len(data)*Split_Ratio)) + 1: , :].reset_index(drop=True)

In [10]:
Train_df

,Tweet,Class
0,عاصمہ شیرازی صاحبہ،,Positive
1,بات کہنی ہو کوئی وعدہ نبھانا ہواسے آواز دینی ہ...,Negative
2,آقا صلی اللہ علیہ و آلہ وسلم کے صحابہ صحبت کی ...,Positive
3,انشا ءاللہ بابے دا ڈیرہ سے تا ریخی قافلہ گجرات...,Positive
4,بنوں میں دوستانہ میچ آئی ڈی پیز کی شاہد آفریدی...,Positive
...,...,...
1063,مذہب کی چادر میں بدنما داغ ہیں ۔ ان بدنما داغو...,Negative
1064,اس ملک میں انصاف کا بھی کوئی نظام موجود ہے ۔۔؟,Negative
1065,انشا جی اٹھو اب کوچ کرو اس شہر میں جی کا لگانا...,Positive
1066,راولپنڈی فتح جنگ میں وین اور ٹینکر کی ٹکر 2 لو...,Negative


In [11]:
Train = dict.fromkeys(classes,[])
Test = dict.fromkeys(classes,[])

for c in classes:
  Train[c] = Train_df.loc[Train_df['Class'] == c ]['Tweet'].tolist()
  Test[c] = Test_df.loc[Test_df['Class'] == c]['Tweet'].tolist()

Reading the given stop words 

In [12]:
#### Read the stop words
# f = open( r'drive/My Drive/NLP/Assignment 5/data/stopwords-ur.txt'  ,'rt', encoding="utf-8-sig" )
# stop_words = f.read().split('\n')

A function that takes the training data set and makes a vocabulary of unique words. It also takes in a bool that determines whether it should include stopwords or not

In [13]:
def makeVocabulary(T , stopwords = False):
  V = set()
  for c in T:
    for text in T[c]:
      words = text.split()
      for word in words:
        if stopwords == True and word in stop_words:
          continue
        V.add(word)
  V = list(V)
  return V

Function that given a set of classes and train dataset, Trains a conditionalProbability model for fake news classification. It also takes in two bools. Bool 'booleanNB' determines whether the boolean Naive Bayes algorithm will be used to train the data set and boolean 'stopwords' determine whether the stopwords are to be considered while training

In [14]:
def TrainMultinomialNB(C, T , booleanNB = False , stopwords = False ):
  V = makeVocabulary(T , stopwords)
  Priors = {}
  condProb = {}

  sum_of_priors = 0

  for c in C:
    sum_of_priors += len(T[c])
  
  for c in C:
    Priors[c] = len(T[c])/sum_of_priors

    doc = []
    for text in T[c]:
      templist = text.split()

      if booleanNB == True:
        doc += list(set(text.split()))
      else:
        doc += text.split()

    count_words = dict.fromkeys( V , 0 )
    for word in doc:
      if word in count_words.keys():
        if stopwords == True and word in stop_words:
          continue
        count_words[word] += 1

    
    for word in V:
      condProb[word , c] = ( count_words[word] + 1 ) / ( len(doc) + len(V) ) 

  return V , Priors , condProb

A function that takes in the vocabulary, the classes, the condtitional probabilty model and the test text and determines in which class the test text belongs. It also takes in a bool to specify whether it should consider stopwords or not 

In [15]:
def ApplyMultinomialNB(C, V, prior, condprob, t , stopwords = False):
  words_from_text = t.split()
  score = dict.fromkeys(C , 0)
  
  for c in C:
    score[c] = math.log(prior[c])
    for word in words_from_text:
      if stopwords == True and word in stop_words:
        continue
      if (word , c) in condprob.keys():
        score[c] += math.log(condprob[word , c])

  if (score['Positive'] > score['Negative']):
    return 'Positive'
  else:
    return 'Negative'

# Train and test the **Naive Baise Model with Stop Words** on the test set

In [16]:
### Training Naive Baise Model with Stop Words
vocab , priors , cond_Prob = TrainMultinomialNB(classes , Train , booleanNB = False , stopwords = False)

In [19]:
results_of_NB_withStopWords = confusion_matrix()
for c in classes:
  for text in Test[c]:
    result = ApplyMultinomialNB(classes, vocab, priors, cond_Prob, text , stopwords = False)
    results_of_NB_withStopWords.update_matrix(c , result)
  
results_of_NB_withStopWords.results()

Accuracy :  65.91760299625467
Precision:  60.8
Recall   :  64.40677966101694
F1 score :  62.55144032921811
